In [1]:
import sys
sys.path.append('./Ray_finrl/findrl_ray/finenv')
from finenv.env_stocktrading import StockTradingEnv
from finenv.preprocessors import FeatureEngineer, data_split
import pandas as pd
import pickle 
# load the DataFrame from a pickle file
#df = pd.read_pickle('dataset/processed.pkl')
trade = pd.read_csv('dataset/trade_data.csv')
trade = trade.set_index(trade.columns[0])
trade.index.names = ['']

In [2]:
INDICATORS = ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma']

In [3]:
import psutil
import ray
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total
from ray.tune.registry import register_env
from gymnasium.wrappers import EnvCompatibility

In [4]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 80, State Space: 801


In [5]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 500,
    "initial_amount": 3000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [6]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None, risk_indicator_col= 'vix', **env_kwargs)

In [7]:
def env_creator_trading(env_config):
    # env_config is passed as {} and defaults are set here
    df = env_config.get('df', trade)  # changed from the training environment
    hmax = env_config.get('hmax', 500)
    initial_amount = env_config.get('initial_amount', 3000000)
    num_stock_shares = env_config.get('num_stock_shares', [0] * stock_dimension)
    buy_cost_pct = env_config.get('buy_cost_pct', buy_cost_list)
    sell_cost_pct = env_config.get('sell_cost_pct', sell_cost_list)
    state_space = env_config.get('state_space', 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension)
    stock_dim = env_config.get('stock_dim', stock_dimension)
    tech_indicator_list = env_config.get('tech_indicator_list', INDICATORS)
    action_space = env_config.get('action_space', stock_dimension)
    reward_scaling = env_config.get('reward_scaling', 1e-4)
    # specific for trading
    #turbulence_threshold = env_config.get('turbulence_threshold', 70)
    #risk_indicator_col = env_config.get('risk_indicator_col', 'vix')
    return EnvCompatibility(StockTradingEnv(
        df=df,
        hmax=hmax,
        initial_amount=initial_amount,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=buy_cost_pct,
        sell_cost_pct=sell_cost_pct,
        state_space=state_space,
        stock_dim=stock_dim,
        tech_indicator_list=tech_indicator_list,
        action_space=action_space,
        reward_scaling=reward_scaling,
        #turbulence_threshold=turbulence_threshold,
        #risk_indicator_col=risk_indicator_col
    ))

In [8]:
from ray.rllib.agents import ppo
ray.shutdown()
ray.init(num_cpus=8, num_gpus=0)
config = ppo.PPOConfig()
config = config.environment(env_config={'hmax':500,'initial_amount':300000})
config = config.training(gamma=0.9, lr=0.001, kl_coeff=0.3)  
config = config.resources(num_gpus=0)
config = config.rollouts(num_rollout_workers=0) 
config = config.framework(framework="torch")
config = config.exploration(explore=False)
config['explore'] = False
config['seed'] = 0
config["model"]["fcnet_hiddens"] = [1024, 256, 128,16]

2023-04-02 22:02:13,996	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


In [9]:
# registering the environment to ray
register_env("finrl", env_creator_trading)
#trainer = config.build(env="finrl")
# define trainer
#trainer = ppo.PPOTrainer(env='finrl', config=config)
#trainer = config.build(env="finrl") 

In [10]:
#load model
#cwd_checkpoint = 'model/4fcnet/checkpoint_000400'
cwd_checkpoint = 'model/checkpoint_000026'
#trainer.restore(cwd_checkpoint)

from ray.rllib.algorithms.algorithm import Algorithm
trainer = Algorithm.from_checkpoint(cwd_checkpoint)


2023-04-02 22:02:15,236	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=30237) 2023-04-02 22:02:22,890	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=30237) 2023-04-02 22:02:22,890	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(RolloutWorker pid=30244) /home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
(RolloutWorker pid=30244)   logger.warn("Casting input x to numpy array.")
(RolloutWorker pid=30237) /home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasi

In [1]:
import time 
def DRL_prediction(model, environment):
    start = time.time()
    """make a prediction"""
    state = environment.reset()
    for i in range(len(environment.df.index.unique())):
        action = model.compute_single_action(state, explore=False)
        state, reward, done, _ = environment.step(action)
        if i == (len(environment.df.index.unique()) - 1):
            account_memory = environment.save_asset_memory()
            actions_memory = environment.save_action_memory()
        if done:
            break
    print("Test Finished! ")
    print((time.time()-start),'s')
    return account_memory, actions_memory

In [12]:
df_account_value, df_actions = DRL_prediction(model=trainer, environment = e_trade_gym)
print(df_account_value.iloc[-1]['account_value'])
df_actions.sum()

Test Finished!
7.737316131591797 s
4909397.815860947


AAL       0
AAPL    168
ADBE      0
ADI     104
ADP     408
       ... 
VRTX      0
WBA     711
WDC      48
WYNN     64
XEL     112
Length: 80, dtype: int64

In [ ]:
for _ in range(3):
    df_account_value, df_actions = DRL_prediction(model=trainer, environment = e_trade_gym)
    print(df_account_value.iloc[-1]['account_value'])

Test Finished!
7.696302890777588 s
4533666.082579371


In [ ]:
#Total Trades
counts = (df_actions > 0).sum()
tdays = int(df_actions.shape[0])
print(f'Trades:{counts.sum()}\nTrade Days:{tdays}\nTrades Per Day:{counts.sum()/tdays:.2f}')

In [ ]:
df_actions.sum()

In [ ]:
import datetime
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./results/"+"perf_stats_all"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

#stats = backtest_stats(baseline_df, value_col_naxme = 'close')

In [ ]:
print("==============Compare to NDX===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^NDX', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])